In [ ]:
!pip install openai
!pip install sounddevice

In [ ]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

print(OPENAI_API_KEY)

In [ ]:
import sounddevice
from scipy.io.wavfile import write
sampleRate = 44100
print ("recording...")

audio = sounddevice.rec(int(12 * sampleRate), sampleRate, channels=1)
sounddevice.wait()
write("recording.wav", sampleRate, audio)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

file_path = "./proven_recording.wav"
# file_path = "./recording.wav"

with open(file_path, "rb") as audio_file:
    response = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        language="en",
        response_format="text"
    )

if response:
    print("Transcription:", response)

transcription = response

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.with_raw_response.create(
    messages=[{
        "role": "user",
        "content": "What is the name of the meme that the user is asking about in the phrase that follows the # character? Emit just the meme name, nothing else. # " + transcription
    }],
    model="gpt-3.5-turbo",
)
print(response.headers.get('X-My-Header'))

completion = response.parse()  # get the object that `chat.completions.create()` would have returned
memeName = completion.choices[0].message.content
print (memeName)

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.with_raw_response.create(
    messages=[{
        "role": "user",
        "content": "The web page at  https://imgflip.com/popular-meme-ids contains a table of meme image IDs and descriptions. What is the ID of the '" + memeName + "' meme? Emit just the ID number, nothing else."
    }],
    model="gpt-3.5-turbo",
)
print(response.headers.get('X-My-Header'))

completion = response.parse()  # get the object that `chat.completions.create()` would have returned
memeNumber = completion.choices[0].message.content
print (memeNumber)

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.with_raw_response.create(
    messages=[{
        "role": "user",
        "content": "In the phrase that follows the # character, What 'top text' does the user want to be on the top of meme? Emit just the top text, nothing else. # " + transcription
    }],
    model="gpt-3.5-turbo",
)
print(response.headers.get('X-My-Header'))

completion = response.parse()  # get the object that `chat.completions.create()` would have returned
topText = completion.choices[0].message.content
print (topText)

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.with_raw_response.create(
    messages=[{
        "role": "user",
        "content": "In the phrase that follows the # character, What text does the user want to put on the bottom of meme? Emit just the text, nothing else. # " + transcription
    }],
    model="gpt-3.5-turbo",
)
print(response.headers.get('X-My-Header'))

completion = response.parse()  # get the object that `chat.completions.create()` would have returned
bottomText = completion.choices[0].message.content
print (bottomText)

In [ ]:
import requests
import urllib

url = "https://api.imgflip.com/caption_image"

params = {
    'username':'natewaddoups',
    'password': 'ruwwjZzJMe4K:bZ',
    'template_id': memeNumber,
    'text0':topText,
    'text1':bottomText
}
response = requests.request('POST',url,params=params).json()
print(response)



In [ ]:
import webbrowser

webbrowser.open(response["data"]["url"])